# This code converts 1.csv file, which contains date, time, force, angle ect.. data, to FullRun.csv. (with assigned images of Front, Top, rear, and Live views). This file is then ready to be opened in the HD_Marking_Tool for electrode location identification.

In [1]:
#import is important
import os
import re
import pandas as pd
import numpy as np

In [31]:
#d= 'D:\Thuy Data Transfer\CMUTvsPZT_Characterization\DataWithScripts\CMUT_Test_06Dec18_newCode\BaseLineNoise_vs_JSRgain_Investigation_3_28_2019'
d= 'D:\Thuy Data Transfer\CMUTvsPZT_Characterization\DataWithScripts\CMUT_Test_06Dec18_newCode\Repeatability Test'
directory= d
os.listdir(directory)
count=0
for root, dir, files in os.walk(directory):
    numfiles= len(files)
    count=count+numfiles
    
print(count)

531


['40dB Gain TX Combine data.zip']
['PZT_GE221_1mm_5mmAway_Gain20dB.JPG', 'PZT_GE2_0.6mm_5mmAway_Gain20dB.JPG', 'PZT_GE35_1.2mm_5mmAway_Gain20dB.JPG', 'Thumbs.db']
['PP_Axis0.fig', 'PP_Axis0.jpg', 'PP_Axis1.fig', 'PP_Axis1.jpg', 'PP_Axis2.fig', 'PP_Axis2.jpg', 'PP_Axis3.fig', 'PP_Axis3.jpg', 'PP_Axis4.fig', 'PP_Axis4.jpg', 'RawDataAtBeginning.fig', 'Thumbs.db', 'TX_pzt_ge21_1mm_21Dec2018.mat']
['TX', 'pzt', 'ge21', '1mm', '21Dec2018.mat']
transmitting test
PZT in keywords
Maker is GE
1mm


In [2]:
#Check to see if front,live,rear,and top files are in the directory
def ckdir (d):
    directory= d
    os.chdir(directory)
    files= os.listdir('.')
    picFiles={}
    if 'Front' in files:
        picFiles['Front']=os.listdir('Front')
    else: 
        print("Couldn't find Front in {}".format(directory))
    if 'Live' in files:
        picFiles['Live']=os.listdir('Live')
    else:
        print("Couldn't find Live in {}".format(directory))    
    if 'Rear' in files:
        picFiles['Rear']=os.listdir('Rear')
    else:
        print("Couldn't find Rear in {}".format(directory))   
    if 'Top' in files:
        picFiles['Top']=os.listdir('Top')
    else:
        print("Couldn't find Top in {}".format(directory)) 

    #Sort the names in each folder
    for n in "Front Live Rear Top".split():
        fname= picFiles[n]
        if not n in picFiles: print ("oops ", n)
        fPat= re.compile(n+'(\d+).bmp')
        fK= [ re.match (fPat,f) for f in fname ]
        if not np.all (fK) : print ("Trouble in {}, name of files does not match or at least one missing".format (n))
        fK = [ (int (m.group (1)), m.group (0)) for m in fK]
        s= sorted(fK)
        picFiles[n]= [ p[-1] for p in s ]   
    
    #search for csv files and number 
    allcsvfile=  [ filename for filename in files if filename.endswith( '.csv' ) ]
    csvfile= allcsvfile[0]
    serial= csvfile.split('.')[0]
    os.chdir('..')
    return picFiles, csvfile, serial

In [3]:
# for the newest data sets received after feb 2018 with all images uncropped plus rear and front images are equal
def newpd (csv, picFiles, rD, serial):
# Creating new column names corresponding with 4 electrodes on each strut and 2 on each shaft
    xCols=[ ['Time','Time'], ['Gram Force SP', 'Fsp'],['Gram Force ACT','Fmeas'],]
    for n in "XY": xCols.append([n+' Axis Pos', n+'pos' ])
    for n in "Theta Alpha".split(): xCols.append([n+' Pos', n])
 
    df= pd.read_csv ("{}/{}".format (rD, csv)) #read the csv file

    cols= df.columns #columns
    for sN in range(4): #4 struts
        for eN in range(4): # 4electrodes
            for cor in "XY": #Front strut
                src= "FSP{} E{} {}".format(sN+1,sN*4+eN+1,cor)
                if not src in cols: continue
                xCols.append( [src, "fs{}e{}{}".format(sN+1,eN+1,cor.lower())])
            for cor in "XY": #Top strut
                src= "Top E{} {}".format(sN*4+eN+1,cor)
                if not src in cols: continue
                xCols.append( [src, "ts{}e{}{}".format(sN+1,eN+1,cor.lower())])
            for cor in "XY": #Rear strut
                src= "RSP{} E{} {}".format(sN+1,sN*4+eN+1,cor)
                if not src in cols: continue
                xCols.append( [src, "rs{}e{}{}".format(sN+1,eN+1,cor.lower())])

    for eN in range(2): #shaft electrodes
        for cor in "XY":#Front shaft
            src= "FShft E{} {}".format(eN+1, cor)
            if not src in  cols: continue
            xCols.append([src, "fshft{}{}".format(eN+1, cor.lower())])
        for cor in "XY":#Top shaft
            src= "Top Shft E{} {}".format(eN+1, cor)
            if not src in  cols: continue
            xCols.append([src, "tshft{}{}".format(eN+1, cor.lower())])
        for cor in "XY":#Rear shaft
            src= "RShft E{} {}".format(eN+1, cor)
            if not src in  cols: continue
            xCols.append([src, "rshft{}{}".format(eN+1, cor.lower())])

    nd= pd.DataFrame (df['Date']) #New dataframe
    for x in xCols:
        nd[x[1]]= df[x[0]] #assigning the values to the right columns
    
    #Creating new columns named Live, Top, Front and Rear and assign the .bmp pics in each row
    for pic in "Live Top Front Rear".split() :
        if len(picFiles[pic])== len(nd): 
            nd[pic]= picFiles[pic]
        else:
            print("lengths mismatch, check {} files and dataFrame lengths, {} pics vs {} lines".format (pic, len(picFiles[pic]), len(nd)))
            return []

    #assigning Serial number:
    if 'serial' in df.columns :
        nd['serial'] = df['serial']
    else :
        nd['serial'] = serial
    return nd

In [4]:
#Renaming files in the FullRun csv
def renamePic (dir, nd):
#dir= 'C:\\Users\\HoangT04\\Documents\\deform\\NewData\\20171030_s5'
    #os.chdir(dir)
    #allcsvfile=  [ filename for filename in os.listdir('.') if filename.endswith( '.csv' ) ]
    #FullRun=  [ filename for filename in allcsvfile if filename.startswith( 'FullRun' ) ]
    #csvfile= FullRun[0]
    #df= pd.read_csv ("{}/{}".format (directory, csvfile))
    newdf = nd.copy()
    
    for rx in range(0,len(newdf)):
        row = newdf.iloc[rx]
        month, date, year = row.Date.split ('/')
        hour, minute, sec = row.Time.split (':')
        stamp = "{:04d}{:02d}{:02d}{:02d}{:02d}{:02d}".format (int (year), int (month), int(date),int(hour), int(minute), int(sec) )    
        for view in "Front Top Rear Live".split() :
            oldName = row[view]
            if (not oldName) or oldName == '0' : continue
            fmat= re.compile(r'\d+') 
            num = re.search(r'\d+',oldName).group(0) #number in old file names, ex: Front230.bmp
            newName = "{:}{:}_{:}.bmp".format (view, num, stamp) #ex:Front230_20171121100309
            newdf.loc[rx,view]= newName #assigning new names into images in csv files
            curdir= os.getcwd() #get current directory
            filename= os.path.join(curdir, dir, view, oldName) #location of old file name
            newfilename= os.path.join(curdir, dir, view, newName) #location of new file name
            folderpath= os.path.join(curdir,dir, view)
            if not oldName in os.listdir(folderpath): 
                print(oldName, 'not in folder: ', folderpath )
                continue
            os.rename (filename,newfilename) #change old names to new names
    return newdf   

In [7]:
os.chdir('C:\\Users\\HoangT04\\Documents\\deform\\HD_Data\\Sample39_Runs17to18') #change current working folder
for run in os.listdir ('.') : # go through all the folders in the directory
    #if not run.startswith ('Run') : continue #only go through ones that start with that date
    print ('checking ', run) 
    (picFiles, csvfile, serial)=ckdir(run) #getting the csv file with the associated pictures
    nd= newpd (csvfile, picFiles, run, serial) #reorganized the csv file into a new file
    if len (nd) == 0 : continue
    print ('run is: ', run)
    newnd = renamePic (run, nd) #renaming the picture files in nd and in original Front,Live,Rear,Top
    if len (newnd) ==0: continue 
    newnd.to_csv("{}/FullRun.csv".format(run), index=False)
print('done!')

checking  Run 17
run is:  Run 17
checking  Run 18
run is:  Run 18
done!


In [5]:
pwd


'C:\\Users\\HoangT04\\Documents\\deform\\HD_codes'

In [6]:
#RUN THIS FOR SPECIFIC FILES
os.chdir('C:\\Users\\HoangT04\\Documents\\deform\\HD_data\\Sample39_RUns1to16\\Run 3') #change current working folder
cwd = os.getcwd()
(picFiles, csvfile, serial)=ckdir(cwd) #getting the csv file with the associated pictures
nd= newpd (csvfile, picFiles, cwd, serial) #reorganized the csv file into a new file
newnd = renamePic (cwd, nd) #renaming the picture files in nd and in original Front,Live,Rear,Top
newnd.to_csv("{}/FullRun.csv".format(cwd), index=False)
print('done!')

done!


In [17]:
os.chdir('C:\\Users\\HoangT04\\Documents\\deform\\HD_data\\Sample 56 - Run 4') 

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\Users\\HoangT04\\Documents\\deform\\HD_data\\Sample 56 - Run 4'

In [9]:
#Counting number of files in certain folder
#filename= "Q:\\AF R&D\\Users\\H Thuy\\2017.10.30_13.09.09_0_FluoroFramesMarking"
filename='Q:\\AF R&D\\AR&T Active Projects\\Hybrid localization\\Design\Mechanical modeling\\Advisor HD\\3 Cams data'
os.chdir(filename)
count=0
for run in os.listdir ('.') :
    if run.startswith('Sample'):
        count=count+1
        
print('{} files in this directory'.format(count))

84 files in this directory


In [11]:
84*71

5964